In [1]:
import geomesa_pyspark 
conf = geomesa_pyspark.configure(jars=['/usr/lib/spark/jars/geomesa-hbase-spark-runtime_2.11-2.1.0-m.2.jar'],
                                 packages=['geomesa_pyspark','pytz'], 
                                 spark_home='/usr/lib/spark/') 
#setAppName('MyTestApp')
conf.get('spark.master') # u'yarn'
from pyspark.sql import SparkSession
spark = ( SparkSession.builder.config(conf=conf).getOrCreate() )


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
111,application_1599127947832_0112,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
params = {    "hbase.zookeepers": "hbase.optix-ons-local:2181",    "hbase.catalog": "ons-historical" } 
feature = "ee" 
ee = ( spark    .read    .format("geomesa")    .options(**params)    .option("geomesa.feature", feature)    .load() )
ee.createOrReplaceTempView("ee")


df1=spark.sql("""select 
mmsi,
dtg,
vessel_type_code,
vessel_type_cargo,
flag_country,
destination,
eta,
draught,
longitude,
latitude,
sog,
rot,
heading,
nav_status_code,
nav_status,
source,
length,
message_type,
vessel_type_main,
vessel_type_sub,
eeid
            from ee where 
           st_intersects(position,st_makeBox2d(st_point(-46.4816,-24.1178),st_point(-46.1763,-23.8787))) AND
           dtg >= cast('2020-01-01 00:00' as timestamp) and   
           dtg < cast('2020-09-01 00:00' as timestamp) and nav_status_code IN (1,5)
           """).write.csv("s3://optix.ons.jupyter/jupyter/dwater/folder/forecast_2")

In [ ]:
df1=spark.sql("""select mmsi, dtg, vessel_type_code, vessel_type_cargo, flag_country, destination, eta, draught, longitude, latitude, sog, rot, heading, nav_status_code, nav_status, source, length, message_type, vessel_type_main, vessel_type_sub, eeid from ee where st_intersects(position,st_makeBox2d(st_point(-46.4816,-24.1178),st_point(-46.1763,-23.8787))) AND dtg >= cast('2020-09-01 00:00' as timestamp) order by dtg asc""").coalesce(1).write.option("header", "true").csv("s3://optix.ons.jupyter/jupyter/dwater/folder/forecast_6")
    
    
    

In [4]:
df1=spark.sql("""select mmsi, dtg, vessel_type_code, vessel_type_cargo, flag_country, destination, eta, draught, longitude, latitude, sog, rot, heading, nav_status_code, nav_status, source, length, message_type, vessel_type_main, vessel_type_sub, eeid from ee where dtg >= cast('2020-09-07 11:00' as timestamp) order by dtg asc""").coalesce(1).write.option("header", "true").csv("s3://optix.ons.jupyter/jupyter/dwater/folder/forecast_7")

In [11]:
df1=spark.sql("""
SELECT
    *
FROM
(
SELECT
    mmsi,
    (
        (
            unix_timestamp(MAX(dtg)) - unix_timestamp('2020-09-07 15:00:00')
        )/60
    ) AS last_transmission,
    max(latitude),
    max(longitude)
FROM
    ee
WHERE
    nav_status_code = 0
and
    dtg >= cast(('2020-09-07 15:00:00') as timestamp)
and
    vessel_type_code IN (
    '60', '61', '62', '63', '64', '65', '66', '67', '68', '69',
    '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
    '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
    '90', '91', '92', '93', '94', '95', '96', '97', '98', '99' 
    )
GROUP BY
    mmsi
ORDER BY
    mmsi
ASC
) t
WHERE
    t.last_transmission >= 10
ORDER BY
    t.last_transmission
DESC
""").coalesce(1).write.option("header", "true").csv("s3://optix.ons.jupyter/jupyter/dwater/folder/rodrigo/transmission")

# .coalesce(1).write.option("header", "true").csv("s3://optix.ons.jupyter/jupyter/dwater/folder/forecast_7")

In [13]:
df1=spark.sql("""
SELECT
    round(latitude, 2) as latitude,
    round(longitude, 2) as longitude,
    count(mmsi)
FROM
    ee
WHERE
    message_type IN ('5') 
and
    dtg >= cast(('2020-09-07 11:00:00') as timestamp)
and
    dtg >= cast(('2020-09-07 12:00:00') as timestamp)
group by
    round(latitude, 2),
    round(longitude, 2)
order by
    round(latitude, 2),
    round(longitude, 2)
asc
""").show(50)

An error was encountered:
Invalid status code '404' from http://ip-10-250-1-253.ec2.internal:8998/sessions/96 with error payload: "Session '96' not found."


In [ ]:
df1=spark.sql("""select count(1),  CAST(dtg AS DATE) FROM
                (select DISTINCT(mmsi), CAST(dtg AS DATE)
            from ee where 
           st_intersects(position,st_makeBox2d(st_point(-46.4816,-24.1178),st_point(-46.1763,-23.8787))) AND
           dtg >= cast('2020-01-01 00:00' as timestamp) and   
           dtg < cast('2020-09-01 00:00' as timestamp) and nav_status_code = 5 ORDER BY dtg, mmsi)
           group by dtg order by dtg""").show(10)

In [11]:
df1.write.csv("s3://optix.ons.jupyter/jupyter/dwater/folder/deb2")

'NoneType' object has no attribute 'write'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'write'



In [ ]:
df1=spark.sql("""select 
position from ee where 
           st_intersects(position,st_makeBox2d(st_point(-46.4816,-24.1178),st_point(-46.1763,-23.8787))) AND
           dtg >= cast('2020-02-01 00:00' as timestamp) and   
           dtg < cast('2020-02-02 00:00' as timestamp) """).show(10)

In [ ]:
df1.write.csv("s3://optix.ons.jupyter/jupyter/dwater/folder/td")

In [ ]:
df1=spark.sql("""select * from ee where 
           st_intersects(position,st_makeBox2d(st_point(-46.4816,-24.1178),st_point(-46.1763,-23.8787))) AND
           dtg > cast('2019-01-01 00:00' as timestamp) and   
           dtg < cast('2019-01-02 00:00' as timestamp) AND (mmsi = 356347000) order by dtg""")

mmsi,imo,dtg,vessel_name,callsign,vessel_type,vessel_type_code,
vessel_type_cargo,vessel_class,length,width,flag_country,flag_code,destination,eta,
draught,position,longitude,latitude,sog,cog,rot,heading,nav_status,nav_status_code,source,
ts_pos_utc,ts_static_utc,dt_pos_utc,dt_static_utc,vessel_type_main,vessel_type_sub,message_type,eeid 

In [ ]:
!hdfs dfs -ls /

In [ ]:
df.coalesce(1).write.csv("/myresults.csv")

In [ ]:
#result set will be very large, show only 10 results for this example
spark.sql(""" select dt_pos_utc, mmsi, vessel_name, longitude, latitude 
                FROM ee WHERE mmsi = 366206000 AND dt_pos_utc > date_add(current_timestamp(), -1) """)

In [ ]:
spark.sparkContext.applicationId

In [ ]:
### média de velocidade
df1=spark.sql(""" SELECT longitude, SUM(sog)/COUNT(*) 
                    FROM ee WHERE 
                    st_intersects(position,st_makeBox2d(st_point(-46,-20),st_point(15,-25))) 
                    AND dtg >= cast('2020-09-01' as DATE) 
                    group by longitude order by longitude""").show(10)